In [1]:
import os
os.environ["DEVELOPMENT"] = "DEVELOPMENT"

In [7]:
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement
from aquacrop.utils import prepare_weather, get_filepath
import torch

In [3]:
path = get_filepath('champion_climate.txt')
wdf = prepare_weather(path)

/Users/danyoung/workspace/aquacrop/.venv/lib/python3.9/site-packages/aquacrop/utils/prepare_weather.py:23: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  weather_df = pd.read_csv(weather_file_path, header=0, delim_whitespace=True)
/Users/danyoung/workspace/aquacrop/.venv/lib/python3.9/site-packages/aquacrop/utils/prepare_weather.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather_df.ReferenceET.clip(lower=0.1, inplace=True

In [4]:
wdf

,MinTemp,MaxTemp,Precipitation,ReferenceET,Date
0,-21.11,3.33,0.0,1.59,1982-01-01
1,-10.00,0.56,0.0,0.86,1982-01-02
2,-11.67,-2.22,0.0,0.72,1982-01-03
3,-12.22,7.22,0.0,1.71,1982-01-04
4,-14.44,-1.11,0.0,0.92,1982-01-05
...,...,...,...,...,...
13509,-7.93,-2.01,0.0,0.52,2018-12-27
13510,-15.52,-5.35,0.0,0.62,2018-12-28
13511,-17.23,2.76,0.0,1.37,2018-12-29
13512,-11.28,10.96,0.0,2.13,2018-12-30


In [25]:
wdf_torch = torch.tensor(wdf[["MinTemp", "MaxTemp", "Precipitation", "ReferenceET"]].values, dtype=torch.float32, device="mps")
wdf_torch = wdf_torch.unsqueeze(0)

In [26]:
wdf_torch.shape

torch.Size([1, 13514, 4])

In [30]:
lstm = torch.nn.LSTM(4, 16, batch_first=True, device="mps")
linear = torch.nn.Linear(16, 4, device="mps")
h0 = torch.randn((1, 1, 16), device="mps")
c0 = torch.randn((1, 1, 16), device="mps")

In [28]:
output, (hn, cn) = lstm(wdf_torch, (h0, c0))

In [47]:
unscaled = linear(hn).squeeze()
torch.sigmoid(unscaled) * 100

tensor([45.4670, 32.7575, 46.3359, 55.7025], device='mps:0',
       grad_fn=<MulBackward0>)

In [53]:
for name, param in lstm.named_parameters():
    print(name, param.shape)

for param in lstm.parameters():
    print(param.shape)

weight_ih_l0 torch.Size([64, 4])
weight_hh_l0 torch.Size([64, 16])
bias_ih_l0 torch.Size([64])
bias_hh_l0 torch.Size([64])
torch.Size([64, 4])
torch.Size([64, 16])
torch.Size([64])
torch.Size([64])
